In [25]:

!pip install --upgrade xee
!pip install rioxarray

In [26]:
import requests
import geopandas as gpd
from shapely.geometry import mapping, shape
from pyproj import CRS, Transformer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import ee
from ee.ee_exception import EEException

## Computing Data Using GEE

In [ ]:
# Get authetication token and sign in to Google Earth Engine
ee.Authenticate()
ee.Initialize()

In [3]:
# signing in using a service account
# cloud_project = "trofmis"
# service_account = "yoda-geospatial-research@yoda-geospatial-research.iam.gserviceaccount.com" # replace with your service account
# gee_key = "/content/auth.json" # replace with your key

# try:
#     ee.Initialize(project=cloud_project, opt_url='https://earthengine-highvolume.googleapis.com')
# except:
#     # ee.Authenticate()
#     # ee.Initialize(project=cloud_project, opt_url='https://earthengine-highvolume.googleapis.com')
#     credentials = ee.ServiceAccountCredentials(service_account, gee_key)
#     ee.Initialize(credentials)

In [4]:
def get_ee_geometry(geometry):
    """This function returns Google Earth engine feature collection"""
    ee_geometry = None
    for geom in geometry["features"]:
        try:
            ee_geom = None
            geom = geom["geometry"]

            if geom["type"] == "Polygon":
                ee_geom = ee.Geometry.Polygon(geom["coordinates"])
            elif geom["type"] == "MultiPolygon":
                ee_geom = ee.Geometry.MultiPolygon(geom["coordinates"])
            elif geom["type"] == "Point":
                ee_geom = ee.Geometry.Point(geom["coordinates"])
            else:
                raise ValueError("Only Points and Polygons are supported.")
            ee_geometry = ee_geometry.union(ee_geom) if ee_geometry else ee_geom

        except EEException:
            log.exception("An error occurred while trying to generate an ee object.")
    return ee_geometry

In [5]:
smoothing_radius = 50

In [6]:
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).multiply(0.0001) \
      .select('B.*') \
      .copyProperties(image, ['system:time_start'])

def renamebandsS2(image):
    """this function used to rename band names for sentinel 2 images"""
    reneamed = image.select(['B3', 'B2', 'B4', 'B8', 'B11', 'B12', 'B5', 'B1',
                             'B6', 'B7', 'B8A', 'B9', 'B10',  ],
     ['Green', 'Blue', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'Red_Edge',
      'Aerosols', 'Red_Edge_2', 'Red_Edge_3', 'Red_Edge_4', 'Water_vapor', 'Cirrus'])
    return reneamed

def NDMI(image):
    """Compute normalized difference moisture index"""
    nmdi = image.expression('(nir-swir)/(nir+swir)', {'nir': image.select(['NIR']), 'swir': image.select(['SWIR1'])}).rename('NDMI')
    return image.addBands(nmdi)


def NDWI(image):
    ndwi = image.expression('(green-nir)/(green+nir)', {'nir': image.select(['NIR']), 'green': image.select(['Green'])}).rename('NDWI')
    return image.addBands(ndwi)
def add_ci(image):
    """
    returns chlorophyl index image
    """
    ci = image.expression(
        '(Red_Edge - red)/(Red_Edge + red)', {'Red_Edge': image.select(['Red_Edge']), 'red': image.select(['RED'])}
    ).rename('CI')

    return image.addBands(ci)
def NDVI(image):
    """This function returns NDVI given image"""
    ndvi = image.normalizedDifference(['NIR', 'RED']).rename('NDVI')
    # colle = image.addBands(ndvi)
    # ndvi_onely = colle.select('ndviS2')
    return image.addBands(ndvi)

def renamebandsL8_sr(image):
    """rename bands for Landsat 8 sensor"""
    renamed = image.select(
        ['SR_B3', 'SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_B4'],
        ['Green', 'Blue', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'Red_Edge'],
    )
    return renamed

def maskL8sr_T1_L2(image):
    # bands = ['SR_B3', 'SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_B4']
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select("QA_PIXEL")
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

def applyFocalMean(image):
  smoothed_image = image.focal_mean(smoothing_radius, 'circle', 'meters');
  return smoothed_image.copyProperties(image, image.propertyNames());

def add_band_count(image):
    return image.set('band_count', image.bandNames().length())

In [7]:
ee_products = {
    'Landsat': {
        'L8':{
            'collection':'LANDSAT/LC08/C02/T1_L2',
            'scale': 30,
            'cloud_mask': 'maskL8sr_T1_L2',
            'rename_bands': 'renamebandsL8_sr',
            'start_date': '2013-04-01',
            'end_date': None,  # to present
        }
    },
    'Sentinel': {
        'S2': {
             'collection': 'COPERNICUS/S2_HARMONIZED',
               'scale': 10,
                'cloud_mask': 'maskS2clouds',
                'rename_bands': 'renamebandsS2',
                'start_date': '2016-01-01',
                'end_date': None,  # to present
        },
        'S1':{

              'collection': 'COPERNICUS/S1_GRD',
              'scale': 10,
             'start_date': '2016-01-01',
              'end_date': None,

     }
    },


}

In [8]:
def getImageCollection(start_date, end_date,  geometry, platform='Sentinel', sensor='S2'):
    """This function returns Google Earth engine image collection
    Args:
        start_date (str): start date of the image collection
        end_date (str): end date of the image collection
        geometry (dict): geometry of the location (geojson feature collection)
        platform (str): platform of the image collection
        sensor (str): sensor of the image collection
    """

    try:
        collection = ee_products[platform][sensor]['collection']

        # scale = ee_products[platform][sensor]['scale']

        maskcloud_func_name = ee_products[platform][sensor].get('cloud_mask', None)

        renamebands_func_name = ee_products[platform][sensor].get('rename_bands', None)


        geometry = get_ee_geometry(geometry)


        image_collection = ee.ImageCollection(collection) \
                            .filter(ee.Filter.bounds(geometry)) \
                            .filter(ee.Filter.date(start_date, end_date))
                             #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70)) \
        """post-processing """
        if sensor == 'S1':

          return image_collection.select(['VV', 'VH']).map(applyFocalMean)
        else:
          maskclouds_func = globals()[maskcloud_func_name]
          renamebands_func = globals()[renamebands_func_name]
          image_collection = image_collection.map(maskclouds_func).map(renamebands_func).map(NDVI).map(NDMI).map(NDWI).map(add_ci)

        return image_collection
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [16]:
def getXarrayImageCollection(ImageCollection, geometry, scale=10):
  """This function returns Google Earth engine image collection"""
  geometry = get_ee_geometry(geometry)
  ds_ = xr.open_dataset(
    ImageCollection,
    engine='ee',
    crs='EPSG:3857',
    scale=scale,
    geometry=geometry,
  )
  return ds_

In [19]:
geojson_small = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -59.55351029230752,
          13.127509051311847
        ],
        "type": "Point"
      }
    }
  ]
}


In [20]:
def mergeImageCollections(S1):
  """Merge image collections one image collection with images from
  s1 being additional bands and by looking at closest dates
  -+ 3 days
  """
  def merger(s2_image):
    s2_date = s2_image.date()
    closest_s1 = S1.filterDate(
        s2_date.advance(-3, 'day'),
        s2_date.advance(3, 'day')
    ).sort('system:time_start').first()

    # Check if a valid S1 image is found
    closest_s1 = ee.Image(ee.Algorithms.If(closest_s1, closest_s1, ee.Image().select([])));

    #Merge the S1 bands with the S2 image (if S1 image exists)
    merged = ee.Image.cat(s2_image, closest_s1).set('system:time_start', s2_image.get('system:time_start'))
    return merged;

  return merger

In [21]:
startDate = '2021-01-01';
endDate= '2021-01-31';

s1_image_collection = getImageCollection(startDate,  endDate, geojson_small, 'Sentinel', 'S1')
s2_image_collection = getImageCollection(startDate,  endDate, geojson_small, 'Sentinel', 'S2')

merged_collection = s2_image_collection.map(mergeImageCollections(s1_image_collection))
collection_with_band_count = merged_collection.map(add_band_count)

# Step 2: Find the maximum number of bands in the collection
max_bands = collection_with_band_count.aggregate_max('band_count').getInfo()
filtered_collection = collection_with_band_count.filter(ee.Filter.eq('band_count', max_bands))

In [24]:
merged_dataset = getXarrayImageCollection(filtered_collection , geojson_small, 10)

In [23]:
merged_dataset

<xarray.Dataset> Size: 688B
Dimensions:      (time: 8, X: 1, Y: 1)
Coordinates:
  * time         (time) datetime64[ns] 64B 2021-01-08T14:39:06.518000 ... 202...
  * X            (X) float64 8B -6.629e+06
  * Y            (Y) float64 8B 1.474e+06
Data variables: (12/19)
    Green        (time, X, Y) float32 32B ...
    Blue         (time, X, Y) float32 32B ...
    RED          (time, X, Y) float32 32B ...
    NIR          (time, X, Y) float32 32B ...
    SWIR1        (time, X, Y) float32 32B ...
    SWIR2        (time, X, Y) float32 32B ...
    ...           ...
    NDVI         (time, X, Y) float32 32B ...
    NDMI         (time, X, Y) float32 32B ...
    NDWI         (time, X, Y) float32 32B ...
    CI           (time, X, Y) float32 32B ...
    VV           (time, X, Y) float32 32B ...
    VH           (time, X, Y) float32 32B ...
Attributes: (12/18)
    date_range:             [1435017600000, 1647993600000]
    description:            <p>Sentinel-2 is a wide-swath, high-resolution, m...
    keywords:               ['copernicus', 'esa', 'eu', 'msi', 'radiance', 's...
    period:                 0
    product_tags:           ['msi', 'radiance']
    provider:               European Union/ESA/Copernicus
    ...                     ...
    type_name:              ImageCollection
    visualization_0_bands:  B4,B3,B2
    visualization_0_max:    3000.0
    visualization_0_min:    0.0
    visualization_0_name:   RGB
    crs:                    EPSG:3857